In [25]:
import sys
from pathlib import Path

PROJECT_ROOT = Path.cwd().parent
sys.path.append(str(PROJECT_ROOT))

In [30]:
pwd

'/Users/takizawakeiya/Desktop/python/git_test/statistical-analysis-toolkit'

In [36]:
cd src

/Users/takizawakeiya/Desktop/python/git_test/statistical-analysis-toolkit/src


/Users/takizawakeiya/Desktop/python/Takizawa-venv/lib/python3.9/site-packages/IPython/core/magics/osm.py:417: UserWarning: using dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [33]:
ls

README.md  doc/       notebook/  sample/    src/       test/


In [38]:
from distribution_moments import (
    DiscreteUniformMoments,
    ContinuousUniformMoments,
    BernoulliMoments
)

In [41]:
dum = DiscreteUniformMoments(0,2)
print(dum.mean())
print(dum.var())

1.0
0.6666666666666666


In [42]:
cum = ContinuousUniformMoments(0,2)
print(cum.mean())
print(cum.var())

1.0
0.3333333333333333


In [44]:
ber = BernoulliMoments(0.2)
print(ber.mean())
print(ber.var())

0.2
0.16000000000000003


In [ ]:
from abc import ABC, abstractmethod

import numpy as np

class BaseDistributionMoments(ABC):
    @abstractmethod
    def mean(self):
        pass

    @abstractmethod
    def var(self):
        pass


class DiscreteUniformMoments(BaseDistributionMoments):
    def __init__(self, lower:float, upper:float):
        self.lower = lower
        self.upper = upper
        if self.lower > self.upper:
            raise ValueError("upper must be greater than lower.")

    def mean(self):
        return (self.upper + self.lower)/2

    def var(self):
        return ((self.upper - self.lower + 1)**2 - 1)/12


class BernoulliMoments(BaseDistributionMoments):
    def __init__(self, p: float):
        self.p = p
        if not 0 <= p <= 1:
            raise ValueError("p must be 0 <= and <= 1.")
    
    def mean(self):
        return self.p

    def var(self):
        return self.p * (1-self.p)


class BinomialMoments(BaseDistributionMoments):
    def __init__(self, n: int, p: float):
        self.n = n
        self.p = p
        if not isinstance(n, int):
            raise ValueError("n must be int.")
        if not 0 <= p <= 1:
            raise ValueError("p must be 0 <= and <= 1.")
    
    def mean(self):
        return self.n * self.p

    def var(self):
        return self.n * self.p*(1-self.p)

# HyperGeometricDistribution
class HyperGeoMoments(BaseDistributionMoments):
    def __init__(self, N: int, M: int, n: int):
        self.N = N
        self.M = M
        self.n = n
        if not isinstance(N, int):
            raise ValueError("N must be int.")
        if not isinstance(M, int):
            raise ValueError("M must be int.")
        if not isinstance(n, int):
            raise ValueError("n must be int.")
        if (n>N or n<=0) or (M>N or M<=0):
            raise ValueError("n and M must be 0 < and <= N")
        if (N<=0):
            raise ValueError("N must be greater than 0.")

    def mean(self):
        return self.n * self.N/self.M

    def var(self):
        return self.n * self.M/self.N * (1 - self.M/self.N) * (self.N-self.n)/(self.N-1)

# Poisson Distribution
class PoissonMoments(BaseDistributionMoments):
    def __init__(self, L):
        self.L = L
        if L<=0:
            raise ValueError("L must be greater than 0")
        
    def mean(self):
        return 1/self.L

    def var(self):
        return 1/self.L


# Grometric Distribution
class GeometricMoments(BaseDistributionMoments):
    def __init__(self, p: float):
        self.p = p
        if not isinstance(p, float):
            raise ValueError("p must be float or 0, 1.")
        if not 0 <= p <= 1:
            raise ValueError("p must be 0 <= and <= 1.")

    def mean(self):
        return (1-self.p)/self.p

    def var(self):
        return (1-self.p)/self.p**2

# Negative Binomial Dostribution
class NBMoments(BaseDistributionMoments):
    def __init__(self, r: int, p: float):
        self.r = r
        self.p = p
        if not isinstance(r, int):
            raise ValueError("r must be int.")
        if not 0 <= p <= 1:
            raise ValueError("p must be 0 <= and <= 1.")
        if r<0:
            raise ValueError("r must be greater than 0.")

    def mean(self):
        return self.r * (1-self.p)/self.p

    def var(self):
        return self.r * (1-self.p)/self.p**2

# Continuous Uniform distribution
class ContinuousUniformMoments(BaseDistributionMoments):
    def __init__(self, lower:float, upper:float):
        self.lower = lower
        self.upper = upper
        if self.lower >= self.upper:
            raise ValueError("upper must be greater than lower.")

    def mean(self):
        return (self.upper + self.lower)/2

    def var(self):
        return (self.upper - self.lower)**2/12


# Normal Distribution
class NormalMoments(BaseDistributionMoments):
    def __init__(self, mu: float, sigma2:float):
        self.mu = mu
        self.sigma2 = sigma2
        if sigma2<=0:
            raise ValueError("σ^2 must be positive.")

    def mean(self):
        return self.mu

    def var(self):
        return self.sigma2